In [8]:
#Libraries used
using Convex
using LinearAlgebra
using MosekTools
using LinearAlgebra

In [2]:
Pkg.add("Random")
Pkg.add("Distributions")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Infinity")

   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [9a3f8284] + Random
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
   Installed Rmath_jll ─────────────── v0.5.1+0
   Installed FillArrays ────────────── v1.13.0
   Installed QuadGK ────────────────── v2.11.0
   Installed StatsFuns ─────────────── v1.3.2
   Installed AliasTables ───────────── v1.1.3
   Installed PtrArrays ─────────────── v1.2.1
   Installed Distributions ─────────── v0.25.111
   Installed Rmath ─────────────────── v0.8.0
   Installed PDMats ────────────────── v0.11.31
   Installed HypergeometricFunctions ─ v0.3.24
    Updating `~/.julia/environments/v1.10/Project.toml`
  [31c24e10] + Distributions v0.25.111
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [66dad0bd] + AliasTables v1.1.3
  [31c24e10] + Distributions v0.25.111
  [1a297f60] + FillArrays v1.13.0
  [34004b35] + HypergeometricFunctions v0.3.24
  [90014a1f] + PDMat

In [1]:
using Pkg
Pkg.add("NBInclude")
Pkg.add("Convex")
Pkg.add("LinearAlgebra")
Pkg.add("MosekTools")
Pkg.add("PyPlot")
Pkg.add("StatsBase")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed NBInclude ─ v2.4.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [0db19996] + NBInclude v2.4.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [0db19996] + NBInclude v2.4.0
Precompiling project...
  ✓ NBInclude
  1 dependency successfully precompiled in 1 seconds. 15 already precompiled.
   Resolving package versions...
   Installed Bzip2_jll ──────────── v1.0.8+1
   Installed IrrationalConstants ── v0.2.2
   Installed DiffRules ──────────── v1.15.1
   Installed BenchmarkTools ─────── v1.5.0
   Installed SpecialFunctions ───── v2.4.0
   Installed DiffResults ────────── v1.1.0
   Installed MutableArithmetics ─── v1.4.6
   Installed CodecBzip2 ─────────── v0.8.4
   Installed StaticArraysCore ───── v1.4.3
   Installed OrderedCollections ─── v1.6.3
   Installed TranscodingStreams ─── v0.11.2
   Installed AbstractTrees ──────── v0.4.5
   Installed LDLFactorization

In [6]:
function lqrsdp(A, B1, B, Q, R)
    n = size(A)[1]
    m = size(B)[2]
    epsilon = 1e-5
    
    W = Semidefinite(m+n,m+n)
    
    problem = minimize(tr(Q*W[(m+1):end,(m+1):end]) + tr(R*W[begin:m,begin:m]))
    problem.constraints += [(A*W[(m+1):end,(m+1):end] - B*W[begin:m,(m+1):end]) + transpose(A*W[(m+1):end,(m+1):end] - B*W[begin:m,(m+1):end]) + B1*transpose(B1) <= 0]
    problem.constraints += [ W[(m+1):end,(m+1):end] - epsilon*I(n) in :SDP]
    problem.constraints += [W in :SDP]
    
    solve!(problem, Mosek.Optimizer #=MosekSolver(verbose = true)=#)
    problem.status
    problem.optval
    
    W_eval = W.value
    Xd = W_eval[(m+1):end,(m+1):end]
    Zd = W_eval[begin:m,(m+1):end]

    K = Zd*inv(Xd)

    return K
end

lqrsdp (generic function with 1 method)

In [ ]:
function system_model(N, AV_number, alpha, beta, v_max, s_st,s_go, s_star, gamma_s, gamma_v, gamma_u)
    alpha1 = @. alpha * v_max / 2 * pi / (s_go - s_st) * sin(pi * (s_star - s_st) / (s_go - s_st))
    alpha2 = @. alpha + beta
    alpha3 = @. beta

    C1 = [0 -1; 0 0] 
    C2 = [0 1; 0 0] 
    pos1 = 1
    pos2 = N

    A = zeros(2 * N, 2 * N)

    for i = 1:(N-1)
        A[(2 * i - 1): (2 * i), (2 * pos1 - 1): (2 * pos1)] = [0 -1; alpha1[i,1] -alpha2[i,1]] #np.array([[0, -1], [alpha1[i - 1][0], -alpha2[i - 1][0]]])
        A[(2 * i - 1): (2 * i), (2 * pos2 - 1): (2 * pos2)] = [0 1; 0 alpha3[i,1]] #np.array([[0, 1], [0, alpha3[i - 1][0]]])
        pos1 = pos1 + 1
        pos2 = mod(pos2 + 1, N)
    end

    A[(2 * N - 1): (2 * N), (2 * pos1 - 1): (2 * pos1)] = C1
    A[(2 * N - 1): (2 * N), (2 * pos2 - 1): (2 * pos2)] = C2
    
    # Controller
    Q = zeros(2 * N, 2 * N)
    for i = 1:N
        Q[2 * i - 1, 2 * i - 1] = gamma_s
        Q[2 * i, 2 * i] = gamma_v
    end
    
    B2 = zeros(2 * N, AV_number)
    B2[2 * N, AV_number] = 1

    if AV_number == 2
        AV2_Index = Int64(floor(N / 2))
        A[(2 * AV2_Index - 1): (2 * AV2_Index), (2 * AV2_Index - 1): (2 * AV2_Index)] = C1
        A[(2 * AV2_Index - 1): (2 * AV2_Index), (2 * AV2_Index - 3): (2 * AV2_Index - 2)] = C2
        B2[2 * AV2_Index, 1] = 1
    end
        
    B1 = zeros(2 * N, N)
        
    for i = 1:N
        B1[2 * i, i] = 1
    end
        
    R = gamma_u .* I(AV_number)

    return A,B1,B2,Q,R
end

In [ ]:
function pattern_generation(N, AV_number, CR)
    if AV_number == 1
        
        K_Pattern = zeros(1, 2 * N)

        for i=1:CR
            K_Pattern[1, 2 * i - 1: 2 * i] = [1 1]
        end
        
        for i=(N-CR):(N-1)
            K_Pattern[1, 2 * i - 1: 2 * i] = [1 1]
            K_Pattern[1, 2 * N - 1: 2 * N] = [1 1]
        end
        
    end

    if AV_number == 2
        if CR >= N - floor(N / 2)
            K_Pattern = ones(2, 2 * N)
        else
            
            K_Pattern = zeros(2, 2 * N)
            
            # row 1
            for i=Int64(floor(N / 2) - CR):(Int64(floor(N / 2) + CR + 1)-1)
                K_Pattern[1, 2 * i - 1 : 2 * i] = [1 1]
            end
            
            # row 2
            for i=1:(CR)
                K_Pattern[2, 2 * i - 1 : 2 * i] = [1 1]
            end
            
            for i=(N - CR):(N-1)
                K_Pattern[2, 2 * i - 1 : 2 * i] = [1 1]
            end
            
            K_Pattern[2,2 * N - 1: 2 * N] = [1 1]
        end
    end
    return K_Pattern
end

In [ ]:
function optsi(A,B1,B2,K_Pattern,Q,R)
    # For a given pattern of K, calculate the optimal feedback gain using sparsity invirance
    # A: system matrix;  B1: distrubance matrix;  B2: control input matrix;

    n = size(A)[1]
    m = size(B2)[2] # number of driver nodes
    epsilon = 1e-5;

    Tp = K_Pattern
    Rp = pattern_invariance(Tp)

    W = Semidefinite(m+n,m+n)

    problem = minimize(tr(Q*W[(m+1):end,(m+1):end]) + tr(R*W[begin:m,begin:m]))

    for i = 1:n
        for j = i:n
            if Rp[i,j]==0
                problem.constraints += [W[m+i,m+j] == 0]
                problem.constraints += [W[m+j,m+i] == 0]
            end
        end
    end

    for i = 1:m
        for j = 1:n
            if Tp[i,j] ==0
                problem.constraints += [W[i,m+j] == 0]
                problem.constraints += [W[m+j,i] == 0]
            end
        end
    end

    problem.constraints += [(A*W[(m+1):end,(m+1):end] - B2*W[begin:m,(m+1):end]) + transpose(A*W[(m+1):end,(m+1):end] - B2*W[begin:m,(m+1):end]) + B1*transpose(B1) <= 0]
    problem.constraints += [ W[(m+1):end,(m+1):end] - epsilon*I(n) in :SDP]
    problem.constraints += [W in :SDP]

    info = solve!(problem, Mosek.Optimizer #=MosekSolver(verbose = true)=#)
    problem.status
    problem.optval

    W_eval = W.value
    X1 = W_eval[(m+1):end,(m+1):end]
    for i=1:n
        for j=i:n
            if Rp[i,j] == 0
                X1[i,j] = 0
                X1[j,i] = 0
            end
        end
    end

    Z1 = W_eval[begin:m,(m+1):end]
    for i=1:m
        for j=1:n
            if Tp[i,j] == 0
                Z1[i,j] = 0
            end
        end
    end
    
    K_Opt = Z1 * inv(X1)
                            
    return K_Opt, info
end

In [1]:
function pattern_invariance(S)
    #Generate a maximally sparsity-wise invariant (MSI) subplace with respect to X
    # See Section IV of the following paper
    # "On Separable Quadratic Lyapunov Functions for Convex Design of Distributed Controllers"

    m = size(S)[1]
    n = size(S)[2]
    X = ones(n,n)
#print(m, " ",n)
    for i = 1:m
        for k = 1:n
            if S[(((i-1)<1) ? end-(i-1) : (i-1)),(((k-1)<1) ? end-(k-1) : (k-1))] == 0
                for j = 1:n
                    if S[(((i-1)<1) ? end-(i-1) : (i-1)),(((j-1)<1) ? end-(j-1) : (j-1))] == 1
                        X[(((j-1)<1) ? end-(j-1) : (j-1)),(((k-1)<1) ? end-(k-1) : (k-1))] = 0
                    end
                end
            end
        end
    end
    
    # symmetric par
    Xu = triu(transpose(conj.(X))) .* triu(X)
    X = Xu .+ transpose(conj.(Xu))
    
    for i=1:size(X)[1]
        for j=1:size(X)[1]
            if(X[i,j] != 0)
                X[i,j] = 1
            end
        end
    end
    
    
    return X
end

pattern_invariance (generic function with 1 method)